# Ticket SITCOM-1167
Craig Lage - 17-Jan-24 

This notebook is to investigate the time delay response of the M1M3 force actuators.  The easiest way to do this is with the bump test results, since here there is a well defined applied force.  The notebook works by taking the applied force and the measured force, then finding the delay which minimizes the difference between these two forces.


## Prepare the notebook

In [ ]:
# Directory to store the data
from pathlib import Path
data_dir = Path("./plots")
data_dir.mkdir(exist_ok=True, parents=True)

start = "2024-01-04T10:00:00"
end = "2024-01-04T11:15:00"

In [ ]:
import sys, time, os, asyncio, glob
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from scipy.optimize import minimize
from astropy.time import Time, TimeDelta
from lsst.ts.xml.tables.m1m3 import FATable, FAIndex, force_actuator_from_id, actuator_id_to_index
from lsst_efd_client import EfdClient
from lsst.summit.utils.tmaUtils import TMAEventMaker

In [ ]:
async def plot_bump_test_actuator_delay(fig, bumps, id, index=0):
    """ Plot a visualization of the bump test actuator delay
        Parameters
        ----------
        fig : a matplotlib figure object

        bumps: pandas dataframe
            This is a dataframe containg the bump test status
        
        id: 'int'
            The actuator id desired

        Returns
        -------
        No return, only the fig object which was input
    """
    def matchFunction(params, args):
        # Determines best shift to match up applied and measured forces
        [times, forces, applied_times, applied_forces] = args
        shifted_forces = np.interp(applied_times + params[0], times, forces)
        diff = applied_forces - shifted_forces
        return np.sum(diff * diff)
    
    this_bump = bumps[bumps['actuatorId']==id]
    last_this_bump_index = bumps[bumps['actuatorId']==id].last_valid_index()
    pass_fail = bumps.iloc[bumps.index.get_loc(last_this_bump_index)+1]
    fa = force_actuator_from_id(id)
    primary_bump = f"primaryTest{fa.index}"
    primary_follow = f"primaryCylinderFollowingError{fa.index}"
    primary_force = f"primaryCylinderForce{fa.index}"
    primary_applied = f"zForces{fa.z_index}"
    if fa.actuator_type.name == 'DAA':
        secondary_bump = f"secondaryTest{fa.s_index}"
        secondary_force = f"secondaryCylinderForce{fa.s_index}"
        secondary_follow = f"secondaryCylinderFollowingError{fa.s_index}"
        secondary_name = fa.orientation.name
        if fa.orientation.name in ['X_PLUS', 'X_MINUS']:
            secondary_applied = f"xForces{fa.x_index}"
        elif fa.orientation.name in ['Y_PLUS', 'Y_MINUS']:
            secondary_applied = f"yForces{fa.y_index}"
    else:
        secondary_name = None
        secondary_force = None
        secondary_follow = None
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Bump Test Response Delay. Actuator ID {id}", fontsize=18)
    plot_start = this_bump[this_bump[primary_bump]==2]['timestamp'].values[index] - 1.0
    plot_end = plot_start + 14.0 
    start = Time(plot_start, format='unix_tai', scale='tai')
    end = Time(plot_end, format='unix_tai', scale='tai')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             [primary_force, primary_follow, 'timestamp'], start.utc, end.utc)

    applied_forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", \
                                         [primary_applied, 'timestamp'], start.utc, end.utc)
    
    timestamp = forces.index[0].isoformat().split('.')[0]
    plt.suptitle(f"Bump Test Time Delays. Actuator ID {id}\n {timestamp}", fontsize=18)
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primary_forces = forces[primary_force].values
    primary_errors = forces[primary_follow].values
    primary_applied_forces = applied_forces[primary_applied].values
    applied_times = applied_forces['timestamp'].values - t0

    plot_start -= t0
    plot_end -= t0
    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(applied_times, primary_applied_forces, label='Requested')
    plt.plot(times, primary_forces, label='Measured')
    plt.xlim(plot_start, plot_end)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (N)")
    plt.legend()
    
    args = [times, primary_forces, applied_times, primary_applied_forces]
    param0 = [0.10] # Estimated time shift
    bestShift = minimize(matchFunction, param0, args=args, method='Powell')
    primary_delay = bestShift.x[0] * 1000.0
    shifted_primary_forces = np.interp(applied_times + bestShift.x[0], times, primary_forces)
    
    plt.subplot(2,2,3)
    plt.title("Primary - Z")
    plt.plot(applied_times, primary_applied_forces, label='Requested')
    plt.plot(applied_times, shifted_primary_forces, label='Shifted Measured')
    plt.text(6, 200, f"Delay = {primary_delay:.1f} ms")
    plt.xlim(plot_start, plot_end)
    plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (N)")
    plt.legend()

    if secondary_name is not None:
        plot_start = this_bump[this_bump[secondary_bump]==2]['timestamp'].values[index] - 1.0
        plot_end = plot_start + 14.0
        start = Time(plot_start, format='unix_tai', scale='tai')
        end = Time(plot_end, format='unix_tai', scale='tai')
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                 [secondary_force, secondary_follow, 'timestamp'], start.utc, end.utc)
        applied_forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", \
                                         [secondary_applied, 'timestamp'], start.utc, end.utc)

        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondary_forces = forces[secondary_force].values
        if fa.orientation.name in ['X_MINUS', 'Y_MINUS']:
            secondary_forces *= -1.0

        secondary_errors = forces[secondary_follow].values
        secondary_applied_forces = applied_forces[secondary_applied].values
        applied_times = applied_forces['timestamp'].values - t0

        plot_start -= t0
        plot_end -= t0
        plt.subplot(2,2,2)
        plt.title(f"Secondary - {secondary_name}")
        plt.plot(applied_times, secondary_applied_forces, label='Requested')
        plt.plot(times, secondary_forces / np.sqrt(2.0), label='Measured')
        plt.xlim(plot_start, plot_end)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (N)")
        plt.legend()

        args = [times, secondary_forces / np.sqrt(2.0), applied_times, secondary_applied_forces]
        param0 = [0.10] # Estimated time shift
        bestShift = minimize(matchFunction, param0, args=args, method='Powell')
        secondary_delay = bestShift.x[0] * 1000.0
        shifted_secondary_forces = np.interp(applied_times + bestShift.x[0], times, secondary_forces)
        
        plt.subplot(2,2,4)
        plt.title(f"Secondary - {secondary_name}")
        plt.plot(applied_times, secondary_applied_forces, label='Requested')
        plt.plot(applied_times, shifted_secondary_forces / np.sqrt(2.0), label='Shifted Measured')
        plt.text(6, 200, f"Delay = {secondary_delay:.1f} ms")
        plt.xlim(plot_start, plot_end)
        plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (N)")
        plt.legend()
    
    else:
        secondary_delay = None
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return [primary_delay, secondary_delay]

## First run just one actuator

In [ ]:
%matplotlib inline
client = EfdClient('usdf_efd')
bumps = await client.select_time_series("lsst.sal.MTM1M3.logevent_forceActuatorBumpTestStatus", "*",\
                                        Time(start, scale='utc'), Time(end, scale='utc'))
# The actuator id runs from 101 to 443, as described in 
# Section 2 of https://sitcomtn-083.lsst.io/
id = 212
fig = plt.figure(figsize=(10,10))
[primary_delay, secondary_delay] = await plot_bump_test_actuator_delay(fig, bumps, id)
plt.savefig(str(data_dir / f"Bump_Test_Delays_{id}.png"))

# Now run the whole bump test

In [ ]:
timestamp = bumps.index[0].isoformat().split('.')[0].replace('-','').replace(':','')
pdf = PdfPages(str(data_dir / f"Bump_Test_Delays_{timestamp}.pdf"))
fig = plt.figure(figsize=(10,10))
delay_dict = {}
for index in range(len(FATable)):
    try:
        id = FATable[index].actuator_id
        [primary_delay, secondary_delay] = await plot_bump_test_actuator_delay(fig, bumps, id)
        pdf.savefig(fig)  # saves the current figure into a pdf page
        print(f"Plot for actuator {id} succeeded!")
        plt.clf()
        delay_dict[id] = [primary_delay, secondary_delay]
    except:
        print(f"Plot for actuator {id} failed!")
        delay_dict[id] = [None, None]
        continue
pdf.close()


In [ ]:
for i in range(len(FATable)):
    id = FATable[i].actuator_id
    [primary_delay, secondary_delay] = delay_dict[id]
    print(id, primary_delay, secondary_delay)


# The cell below plots the delay histograms

In [ ]:
primary_delays = []
secondary_delays = []
for index in range(len(FATable)):
    id = FATable[index].actuator_id
    [primary_delay, secondary_delay] = delay_dict[id]
    if primary_delay:
        primary_delays.append(primary_delay)
    if secondary_delay:
        secondary_delays.append(secondary_delay)
primary_delays = np.array(primary_delays)
primary_delays = primary_delays[primary_delays < 500.0]
secondary_delays = np.array(secondary_delays)
secondary_delays = secondary_delays[secondary_delays < 500.0]

isot_timestamp = bumps.index[0].isoformat().split('.')[0]
fig, axs = plt.subplots(1,2, figsize=(10,5))
axs[0].set_title(f"Primary_delays {isot_timestamp}")
axs[0].hist(primary_delays, bins = 20)
axs[0].text(60, 20, f"Mean = {np.mean(primary_delays):.1f} ms")
axs[0].set_xlim(50,150)
axs[0].set_xlabel("Delay (ms)")
axs[1].set_title(f"Secondary_delays {isot_timestamp}")
axs[1].hist(secondary_delays, bins = 20)
axs[1].text(60, 20, f"Mean = {np.mean(secondary_delays):.1f} ms")
axs[1].set_xlim(50,150)
axs[1].set_xlabel("Delay (ms)")
plt.savefig(str(data_dir / f"Bump_Test_Delay_Histograms_{timestamp}.png"))

# The cell below plots a heat map of the bump test delays

In [ ]:
types = [['SAA','NA', 'o', 'Z'], ['DAA','Y_PLUS', '^', 'Y_PLUS'], ['DAA','Y_MINUS', 'v', 'Y_MINUS'], \
         ['DAA','X_PLUS', '>', 'X_PLUS'], ['DAA','X_MINUS', '<', 'X_MINUS']]

zmin = 80; zmax = 140;
fig = plt.figure(figsize=(8,4))
fig.suptitle(f"Actuator bump test delays {isot_timestamp}\n Colorbar in msec.")
axp = fig.add_axes((0.1, 0.12, 0.35, 0.70))
axs = fig.add_axes((0.55, 0.12, 0.35, 0.70))

axp.set_title("Primary")
axp.set_xlabel("X position (m)")
axp.set_ylabel("Y position (m)")
bad_primarys = "Bad = "
bad_secondarys = "Bad = "
for [type, orient, marker, label] in types:
    xs = []
    ys = []
    zs = []
    for i in range(len(FATable)):
        id = FATable[i].actuator_id
        [primary_delay, secondary_delay] = delay_dict[id]
        x = FATable[i].x_position
        y = FATable[i].y_position
        if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
            if primary_delay > 500:
                bad_primarys += f"{id},"
            xs.append(x)
            ys.append(y)
            zs.append(primary_delay)
    im = axp.scatter(xs, ys, marker='o', c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax, s=50, label=label)
plt.colorbar(im, ax=axp,fraction=0.055, pad=0.02, cmap='RdBu_r') 
axp.text(-4.0, -4.8, bad_primarys)
axs.set_title("Secondary")
axs.set_xlabel("X position (m)")
axp.set_xlim(-5,5)
axp.set_ylim(-5,5)
axs.set_xlim(-5,5)
axs.set_ylim(-5,5)

for [type, orient, marker, label] in types:
    if type == 'SAA':
        continue
    xs = []
    ys = []
    zs = []
    for i in range(len(FATable)):
        id = FATable[i].actuator_id
        [primary_delay, secondary_delay] = delay_dict[id]
        x = FATable[i].x_position
        y = FATable[i].y_position
        if FATable[i].actuator_type.name == type and FATable[i].orientation.name == orient:
            if secondary_delay > 500:
                bad_secondarys += f"{id},"

            xs.append(x)
            ys.append(y)
            zs.append(secondary_delay)
    im = axs.scatter(xs, ys, marker=marker, c=zs, cmap='RdBu_r', vmin=zmin, vmax=zmax, s=50, label=label)
plt.colorbar(im, ax=axs,fraction=0.055, pad=0.02, cmap='RdBu_r') 
axs.text(-4.0, -4.8, bad_secondarys)
plt.savefig(str(data_dir / f"Bump_Test_Delay_Heatmap_{timestamp}.png"))

# Now we try to repeat this on actual slews

In [ ]:
async def plot_slew_actuator_delay(fig, event, id):
    """ Plot a visualization of the actuator delay during a slew
        Parameters
        ----------
        fig : a matplotlib figure object

        event: a TMAEvent
            This determines the start and end times
        
        id: 'int'
            The actuator id desired

        Returns
        -------
        No return, only the fig object which was input
    """
    def matchFunction(params, args):
        # Determines best shift to match up applied and measured forces
        [times, forces, applied_times, applied_forces] = args
        shifted_forces = np.interp(applied_times + params[0], times, forces)
        diff = applied_forces - shifted_forces
        return np.sum(diff * diff)
    
    fa = force_actuator_from_id(id)
    #primary_force = f"primaryCylinderForce{fa.z_index}"
    primary_force = f"zForce{fa.z_index}"
    primary_applied = f"zForces{fa.z_index}"
    if fa.actuator_type.name == 'DAA':
        #secondary_force = f"secondaryCylinderForce{fa.s_index}"
        if fa.orientation.name in ['X_PLUS', 'X_MINUS']:
            secondary_force = f"xForce{fa.x_index}"
            secondary_applied = f"xForces{fa.x_index}"
        elif fa.orientation.name in ['Y_PLUS', 'Y_MINUS']:
            secondary_force = f"yForce{fa.y_index}"
            secondary_applied = f"yForces{fa.y_index}"
    else:
        secondary_force = None
    print(fa.index, fa.z_index, fa.y_index)
    plt.subplots_adjust(wspace=0.3)
    plt.suptitle(f"Slew Response Delay. {event.dayObs} - {event.seqNum} Actuator ID {id}", fontsize=18)
    start = event.begin - TimeDelta(2.0, format='sec')
    end = event.end + TimeDelta(2.0, format='sec')
    forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                             [primary_force, 'timestamp'], start, end)

    applied_forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", \
                                         [primary_applied, 'timestamp'], start, end)
    
    timestamp = forces.index[0].isoformat().split('.')[0]
    times = forces['timestamp'].values
    t0 = times[0]
    times -= t0
    primary_forces = forces[primary_force].values
    primary_applied_forces = applied_forces[primary_applied].values
    applied_times = applied_forces['timestamp'].values - t0

    plt.subplot(2,2,1)
    plt.title("Primary - Z")
    plt.plot(applied_times, primary_applied_forces, label='Requested')
    plt.plot(times, primary_forces, label='Measured')
    #plt.xlim(plot_start, plot_end)
    #plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (N)")
    plt.legend()
    
    args = [times, primary_forces, applied_times, primary_applied_forces]
    param0 = [0.10] # Estimated time shift
    bounds = [(-0.2, 0.2)]
    bestShift = minimize(matchFunction, param0, args=args, bounds=bounds, method='Powell')
    primary_delay = bestShift.x[0] * 1000.0
    shifted_primary_forces = np.interp(applied_times + bestShift.x[0], times, primary_forces)
    
    plt.subplot(2,2,3)
    plt.title("Primary - Z")
    plt.plot(applied_times, primary_applied_forces, label='Requested')
    plt.plot(applied_times, shifted_primary_forces, label='Shifted Measured')
    locs, labels = plt.yticks()
    y_coord = locs[1]
    plt.text(0.0, y_coord, f"Delay = {primary_delay:.1f} ms")
    #plt.xlim(plot_start, plot_end)
    #plt.ylim(-400,400)
    plt.xlabel("Time (seconds)")
    plt.ylabel("Force (N)")
    plt.legend()

    if secondary_force is not None:
        forces = await client.select_time_series("lsst.sal.MTM1M3.forceActuatorData", \
                                                 [secondary_force, 'timestamp'], start, end)
        applied_forces = await client.select_time_series("lsst.sal.MTM1M3.appliedForces", \
                                         [secondary_applied, 'timestamp'], start, end)

        times = forces['timestamp'].values
        t0 = times[0]
        times -= t0
        secondary_forces = forces[secondary_force].values
        secondary_applied_forces = applied_forces[secondary_applied].values
        applied_times = applied_forces['timestamp'].values - t0

        plt.subplot(2,2,2)
        plt.title(f"Secondary - {fa.orientation.name}")
        plt.plot(applied_times, secondary_applied_forces, label='Requested')
        plt.plot(times, secondary_forces, label='Measured')
        #plt.xlim(plot_start, plot_end)
        #plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (N)")
        plt.legend()

        args = [times, secondary_forces, applied_times, secondary_applied_forces]
        param0 = [0.10] # Estimated time shift
        bounds = [(-0.2, 0.2)]
        bestShift = minimize(matchFunction, param0, args=args, bounds=bounds, method='Powell')
        secondary_delay = bestShift.x[0] * 1000.0
        shifted_secondary_forces = np.interp(applied_times + bestShift.x[0], times, secondary_forces)
        
        plt.subplot(2,2,4)
        plt.title(f"Secondary - {fa.orientation.name}")
        plt.plot(applied_times, secondary_applied_forces, label='Requested')
        plt.plot(applied_times, shifted_secondary_forces, label='Shifted Measured')
        locs, labels = plt.yticks()
        y_coord = locs[1]
        plt.text(0.0, y_coord, f"Delay = {secondary_delay:.1f} ms")
        #plt.xlim(plot_start, plot_end)
        #plt.ylim(-400,400)
        plt.xlabel("Time (seconds)")
        plt.ylabel("Force (N)")
        plt.legend()
    
    else:
        secondary_delay = None
        plt.subplot(2,2,2)
        plt.title("No Secondary")
        plt.xticks([])
        plt.yticks([])
        plt.subplot(2,2,4)
        plt.xticks([])
        plt.yticks([])
    return [primary_delay, secondary_delay]

In [ ]:
client = EfdClient('usdf_efd')
dayObs = 20240103
seqNum = 1297
eventMaker = TMAEventMaker()
event = eventMaker.getEvent(dayObs, seqNum)


In [ ]:
%matplotlib inline
# The actuator id runs from 101 to 443, as described in 
# Section 2 of https://sitcomtn-083.lsst.io/
id = 420
fig = plt.figure(figsize=(10,10))
[primary_delay, secondary_delay] = await plot_slew_actuator_delay(fig, event, id)
plt.savefig(str(data_dir / f"Slew_Delays_{dayObs}_{seqNum}_{id}.png"))